In [1]:
from typing import List, NamedTuple
import numpy as np
import jupyros as jr 
import os 
from visualization_msgs.msg import Marker, MarkerArray
from visualization_msgs.msg import Marker
import moveit_commander
from visualization_msgs.msg import Marker
import numpy as np
from scipy.spatial.transform import Rotation
from franka_msgs.msg import ErrorRecoveryAction, ErrorRecoveryActionGoal
import rospy
from gazebo_msgs.srv import (
    GetModelState,
    GetWorldProperties,
    GetModelProperties,
)
from geometry_msgs.msg import PoseStamped


Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [2]:
rospy.init_node("grasp_executor")


In [16]:
import importlib
import src.env.moveit_interface as moveit_interface
importlib.reload(moveit_interface)
import src.env.utils
importlib.reload(src.env.utils)
from src.env.utils import pose_to_list

In [17]:
import moveit_commander
moveit_commander.MoveGroupCommander("panda_manipulator").get_current_joint_values()

[-0.08709680994572455,
 -0.6486011540410326,
 -0.39148927358730923,
 -2.2079177559329297,
 -0.22621135712161689,
 1.6113088232773443,
 0.3982840169424229]

In [32]:
init_pose = [
-0.0001513652608586824,
-0.7856258935003497,
9.991577917034533e-06,
-2.3559505836972523,
2.0776787743592706e-05,
1.5717418653690114,
0.7853832208311591
]
grasp_controller = moveit_interface.GraspExecutor("world", reset_pose=init_pose)
print("Resetted")
grasp_controller.reset() 


Loading static scene information
Set up Franka API. Ready to go!
Resetted


In [33]:
grasp_controller.objects


{'ground_plane': {},
 'table': {},
 'pick_tray': {},
 'place_tray': {},
 'stone': {}}

In [34]:
gazebo_node_name = 'gazebo'
object_name = 'stone'
frame_id = 'world'
get_model_state = rospy.ServiceProxy(f'{gazebo_node_name}/get_model_state', GetModelState)


In [35]:
object_state = get_model_state(object_name, frame_id)
object_pose = object_state.pose
print(object_pose)
pos, rot = pose_to_list(object_pose)
# print(pos, rot)

position: 
  x: 0.00028600034534020436
  y: -0.22197200606608894
  z: 0.46316699963523916
orientation: 
  x: 9.478265936607412e-08
  y: 5.3951634307949206e-09
  z: 1.3941400853136722e-12
  w: 0.9999999999999956


In [36]:
grasp_controller.group.get_current_pose()

header: 
  seq: 0
  stamp: 
    secs: 3648
    nsecs:  18000000
  frame_id: "world"
pose: 
  position: 
    x: 0.00720302524581654
    y: -5.689977424501939e-05
    z: 0.48726680088838087
  orientation: 
    x: 0.999999445881389
    y: -9.968325514353809e-05
    z: 0.0010478476685543661
    w: 1.776026755888041e-05

In [37]:
grasp_controller.grasp(np.array(pos) + np.array([0,0,0.01], dtype=float), np.array([1, 0, 0, 0], dtype=float))
grasp_controller.move_to_pose(np.array(pos) + np.array([0,0,0.1], dtype=float), np.array([1,0,0,0], dtype=float))

Moving to pre-grasp pose
Moved to pre grasp. Remmoving object
Moving to grasp pose
Closing gripper


True

In [24]:
from moveit_msgs.msg import RobotTrajectory, Grasp

## RVIZ markers

In [12]:
jr.publish('/end_effector_pose', PoseStamped)

In [13]:
# pos = [0.15 + 0.3, 0.10, 0.6]
# ori = np.array([0, -0.923, 0.382, 0])
# ori = ori / np.linalg.norm(ori)
# print("Moving to", pos)